In [0]:
sc

SparkContext 

 Spark UI 

 
 Version 
 v3.1.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
import pandas as pd

In [0]:
user_rdd = sc.textFile("/FileStore/tables/users-1.dat")
print(user_rdd.take(2))

['1::F::1::10::48067', '2::M::56::16::70072']


In [0]:
movie_rdd = sc.textFile("/FileStore/tables/movies.dat")
print(movie_rdd.take(2))

["1::Toy Story (1995)::Animation|Children's|Comedy", "2::Jumanji (1995)::Adventure|Children's|Fantasy"]


In [0]:
rat_rdd = sc.textFile("/FileStore/shared_uploads/kartikbhatnagar15@gmail.com/rating_-1.dat")
print("userid::movieId::Rating::Time")
print(rat_rdd.take(2)) 

userid::movieId::Rating::Time
['1::1193::5::978300760', '1::661::3::978302109']


In [0]:
%sh git clone https://github.com/srjsunny/Movie_Lens_Data-Analysis/tree/master/dataSet --depth 1 --branch=master /dbfs/FileStore/tables/


Cloning into '/dbfs/FileStore/tables'...
fatal: repository 'https://github.com/srjsunny/Movie_Lens_Data-Analysis/tree/master/dataSet/' not found


In [0]:
map1 =movie_rdd.map(lambda x : x.split("::")).map(lambda x : (x[0],x[1]))
map1.takeOrdered(6)

Out[61]: [('1', 'Toy Story (1995)'),
 ('10', 'GoldenEye (1995)'),
 ('100', 'City Hall (1996)'),
 ('1000', 'Curdled (1996)'),
 ('1001', "Associate, The (L'Associe)(1982)"),
 ('1002', "Ed's Next Move (1996)")]

In [0]:
#(movie_id , 1)
map2 = rat_rdd.map(lambda x:(x.split("::")[1],1)).reduceByKey(lambda x,y : x+y)
print("MovieId , Totalcount")
map2.take(6)

MovieId , Totalcount
Out[82]: [('914', 164),
 ('3408', 332),
 ('2355', 387),
 ('1197', 548),
 ('2804', 310),
 ('595', 272)]

In [0]:

join1 = map2.leftOuterJoin(map1)
print("[movieId,(Total_Count,movie_name)]")
(join1.take(6))


[movieId,(Total_Count,movie_name)]
Out[83]: [('3408', (332, 'Erin Brockovich (2000)')),
 ('2355', (387, "Bug's Life, A (1998)")),
 ('1197', (548, 'Princess Bride, The (1987)')),
 ('3105', (181, 'Awakenings (1990)')),
 ('2797', (322, 'Big (1988)')),
 ('2321', (294, 'Pleasantville (1998)'))]

In [0]:
#now we have both rdd of rating and movie joined we will make (movieName,1) tuple
reducer1  = join1.map(lambda x : (x[1][1],x[1][0]) )
top10Reducer = reducer1.takeOrdered(10,lambda y:-y[1])
top10Reducer
#Note the orignal data contained 10 Lakh ratings,here due to storage constraint only 2 lakhs ratings are taken

Out[92]: [('American Beauty (1999)', 877),
 ('Star Wars: Episode V - The Empire Strikes Back (1980)', 711),
 ('Star Wars: Episode IV - A New Hope (1977)', 704),
 ('Jurassic Park (1993)', 704),
 ('Star Wars: Episode VI - Return of the Jedi (1983)', 696),
 ('Saving Private Ryan (1998)', 659),
 ('Terminator 2: Judgment Day (1991)', 644),
 ('Matrix, The (1999)', 636),
 ('Men in Black (1997)', 636),
 ('Silence of the Lambs, The (1991)', 615)]

In [0]:
top10Movies = pd.DataFrame(top10Reducer,columns=["Movie Name","No. of Ratings"])
top10Movies.index =(top10Movies.index)+1
top10Movies

,Movie Name,No. of Ratings
1,American Beauty (1999),877
2,Star Wars: Episode V - The Empire Strikes Back...,711
3,Star Wars: Episode IV - A New Hope (1977),704
4,Jurassic Park (1993),704
5,Star Wars: Episode VI - Return of the Jedi (1983),696
6,Saving Private Ryan (1998),659
7,Terminator 2: Judgment Day (1991),644
8,"Matrix, The (1999)",636
9,Men in Black (1997),636
10,"Silence of the Lambs, The (1991)",615


In [0]:
#from kpi 1 in map2 we have count of  ratings for each movie , now we want sum of ratings and then we will join it together to findthe average
map3 = rat_rdd.map(lambda x:(x.split("::"))).map(lambda y:( y[1],int(y[2]))).reduceByKey(lambda x,y : x+y)
print("MovieId , RatingCount")
map3.take(6)

MovieId , RatingCount
Out[117]: [('914', 693),
 ('3408', 1264),
 ('2355', 1493),
 ('1197', 2357),
 ('2804', 1315),
 ('595', 1068)]

In [0]:
join2 = map3.leftOuterJoin(map2)
print("movieId , (Rating Sum ,RatingCount)")
join2.take(6)


movieId , (Rating Sum ,RatingCount)
Out[126]: [('914', (693, 164)),
 ('3408', (1264, 332)),
 ('2355', (1493, 387)),
 ('1197', (2357, 548)),
 ('2804', (1315, 310)),
 ('595', (1068, 272))]

In [0]:
#now we need to take only those   movies for which atleat 40 ratings are done
atleat40 = join2.filter(lambda y : (y[1][1] > 39))
atleat40.take(6)

Out[127]: [('914', (693, 164)),
 ('3408', (1264, 332)),
 ('2355', (1493, 387)),
 ('1197', (2357, 548)),
 ('2804', (1315, 310)),
 ('595', (1068, 272))]

In [0]:
#making (movieId, avg.Rating) tuple
avg_rating = atleat40.map(lambda row : (row[0],round(float(row[1][0]/row[1][1]) ,5)))#reducer 3 in
avg_rating.take(6)

Out[134]: [('914', 4.22561),
 ('3408', 3.80723),
 ('2355', 3.85788),
 ('1197', 4.30109),
 ('2804', 4.24194),
 ('595', 3.92647)]

In [0]:
movie_Avg_rat = avg_rating.leftOuterJoin(map1).map(lambda y:(y[1][1],y[1][0]))
print("MovieId,(Avg. rating,Movie Name)")
top20=movie_Avg_rat.takeOrdered(20 , lambda y: -y[1])
top20

MovieId,(Avg. rating,Movie Name)
Out[143]: [('Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954)',
  4.64706),
 ('Paths of Glory (1957)', 4.6),
 ("Schindler's List (1993)", 4.54643),
 ('Wrong Trousers, The (1993)', 4.5354),
 ('Usual Suspects, The (1995)', 4.53483),
 ('Wallace & Gromit: The Best of Aardman Animation (1996)', 4.53271),
 ('Shawshank Redemption, The (1994)', 4.50478),
 ('Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)', 4.5045),
 ('Godfather, The (1972)', 4.50316),
 ('Third Man, The (1949)', 4.49558),
 ('Yojimbo (1961)', 4.49153),
 ('Close Shave, A (1995)', 4.49091),
 ('Jean de Florette (1986)', 4.48837),
 ('Raiders of the Lost Ark (1981)', 4.47913),
 ('Rear Window (1954)', 4.47012),
 ('Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)',
  4.46732),
 ('It Happened One Night (1934)', 4.45349),
 ('To Kill a Mockingbird (1962)', 4.44395),
 ('Maltese Falcon, The (1941)', 4.43277),
 ('400 Blows, The (Les Quatre cents coups) (1959)', 4.42

In [0]:
#For better Visualisation lets view it in Pandas data frame
kpi2 = pd.DataFrame(top20, columns=["Movie","Average Ratings"])
kpi2.index = (kpi2.index)+1
kpi2

,Movie,Average Ratings
1,Seven Samurai (The Magnificent Seven) (Shichin...,4.64706
2,Paths of Glory (1957),4.60000
3,Schindler's List (1993),4.54643
4,"Wrong Trousers, The (1993)",4.53540
5,"Usual Suspects, The (1995)",4.53483
6,Wallace & Gromit: The Best of Aardman Animatio...,4.53271
7,"Shawshank Redemption, The (1994)",4.50478
8,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),4.50450
9,"Godfather, The (1972)",4.50316
10,"Third Man, The (1949)",4.49558


In [0]:
map4 =movie_rdd.map(lambda x : x.split("::")).map(lambda x : (x[0],x[2]))
print("(movieId , Genere)")
map4.take(6) 

(movieId , Genere)
Out[150]: [('1', "Animation|Children's|Comedy"),
 ('2', "Adventure|Children's|Fantasy"),
 ('3', 'Comedy|Romance'),
 ('4', 'Comedy|Drama'),
 ('5', 'Comedy'),
 ('6', 'Action|Crime|Thriller')]

In [0]:
map5 =user_rdd.map(lambda x : x.split("::")).map(lambda x : (x[0],x[2],x[3]))
print("(userId , Age, Occupation)")
map5.take(6) 

(userId , Age, Occupation)
Out[154]: [('1', '1', '10'),
 ('2', '56', '16'),
 ('3', '25', '15'),
 ('4', '45', '7'),
 ('5', '25', '20'),
 ('6', '50', '9')]

In [0]:
map6 =rat_rdd.map(lambda x : x.split("::")).map(lambda x : (x[0],x[1]+":"+x[2]))
print("(UserId,movieId : Rating)") #instaed of 3 tuple element we kept 2 bcoz the problem was coming during joins
map6.take(6) 

(UserId,movieId , Rating)
Out[192]: [('1', '1193:5'),
 ('1', '661:3'),
 ('1', '914:3'),
 ('1', '3408:4'),
 ('1', '2355:5'),
 ('1', '1197:3')]

In [0]:
def age_group(row):
  
  age= row[1]
  if( int(age) > 50):
    tup = (row[0],"50+ :"+row[2])
    return(tup)
  elif(int(age) <= 50 and int(age) > 35):
    tup = (row[0],"36 to 50 :"+row[2])
    return(tup)
  else:
    tup = (row[0],"18 to 35 :"+row[2])
    return(tup)
  
  
  
  

In [0]:
map7 =map5.map(age_group)
print("(UserId,AgeGroup,Occupation)")
map7.take(6)

(UserId,AgeGroup,Occupation)
Out[190]: [('1', '18 to 35 :10'),
 ('2', '50+ :16'),
 ('3', '18 to 35 :15'),
 ('4', '36 to 50 :7'),
 ('5', '18 to 35 :20'),
 ('6', '36 to 50 :9')]

In [0]:
kp3_join1 = map7.leftOuterJoin(map6)
print("userId,(AgeGroup : Occupation , movieId:Rating)")
kp3_join1.take(6)
#join1 is done for just experimental sake ,futher down it's not use

userId,(AgeGroup : Occupation , movieId:Rating)
Out[205]: [('4', ('36 to 50 :7', '3468:5')),
 ('4', ('36 to 50 :7', '1210:3')),
 ('4', ('36 to 50 :7', '2951:4')),
 ('4', ('36 to 50 :7', '1214:4')),
 ('4', ('36 to 50 :7', '1036:4')),
 ('4', ('36 to 50 :7', '260:5'))]

In [0]:
map8 =rat_rdd.map(lambda x : x.split("::")).map(lambda x : (x[1],x[0]+":"+x[2]))
print("(movieId ,UserId : Rating)") #instaed of 3 tuple element we kept 2 bcoz the problem was coming during joins
map8.take(6) 

(movieId ,UserId : Rating)
Out[206]: [('1193', '1:5'),
 ('661', '1:3'),
 ('914', '1:3'),
 ('3408', '1:4'),
 ('2355', '1:5'),
 ('1197', '1:3')]

In [0]:
usr_gnr = map8.leftOuterJoin(map4).map(lambda y:((y[1][0]).split(":")[0],((y[1][1]+":"+(y[1][0]).split(":")[1]))))
print("userId,(Genre : Rating")
usr_gnr.take(6)

userId,(Genre : Rating
Out[216]: [('1', 'Drama:4'),
 ('5', 'Drama:3'),
 ('6', 'Drama:5'),
 ('9', 'Drama:4'),
 ('10', 'Drama:4'),
 ('15', 'Drama:4')]

In [0]:
kpi3_join2 = usr_gnr.leftOuterJoin(map7)
print("UserId,(Genre:Rating  ,  AgeGroup:Occupation)")
kpi3_join2.takeSample(6,10)


UserId,(Genre:Rating  ,  AgeGroup:Occupation)
Out[219]: [('1289', ('Action|Adventure:4', '36 to 50 :6')),
 ('1328', ('Action:5', '18 to 35 :14')),
 ('956', ('Musical|Romance:4', '18 to 35 :1')),
 ('314', ('Film-Noir|Thriller:5', '50+ :9')),
 ('905', ('Sci-Fi|War:5', '36 to 50 :12')),
 ('1420', ('Western:3', '18 to 35 :4')),
 ('1209', ('Action|Comedy|Western:2', '18 to 35 :7')),
 ('86', ('Thriller:5', '18 to 35 :10')),
 ('710', ('Crime|Drama|Thriller:3', '18 to 35 :20')),
 ('1242', ('Comedy|Romance|Thriller:3', '18 to 35 :0'))]

In [0]:
#now we just want a tuple like (AgeGroup:Occpation:Genre  , Rating)
agor_sum = kpi3_join2.map(lambda y : ( y[1][1]+":"+((y[1][0]).split(":"))[0]   ,        float( (y[1][0].split(":"))[1] ) ))
print("(AgeGroup:Occpation:Genre  , Rating)")
agor_sum.take(6)

(AgeGroup:Occpation:Genre  , Rating)
Out[222]: [('18 to 35 :0:Drama', 4.0),
 ('18 to 35 :0:Action|Adventure|Comedy|Romance', 5.0),
 ('18 to 35 :0:Action|Adventure|Drama|Sci-Fi|War', 5.0),
 ('18 to 35 :0:Comedy|Romance|War', 5.0),
 ('18 to 35 :0:Comedy', 4.0),
 ("18 to 35 :0:Animation|Children's|Comedy|Musical|Romance", 4.0)]

In [0]:
sum = agor.reduceByKey(lambda x,y : x+y)
sum.take(6)

Out[223]: [('18 to 35 :0:Action|Adventure|Comedy|Romance', 506.0),
 ("18 to 35 :0:Animation|Children's|Comedy|Musical|Romance", 166.0),
 ('18 to 35 :0:Drama|Romance', 2747.0),
 ('18 to 35 :0:Comedy|Musical|Romance', 234.0),
 ('18 to 35 :0:Comedy|Romance|Thriller', 234.0),
 ('18 to 35 :0:Action|Adventure|Comedy', 241.0)]

In [0]:
agor_count = kpi3_join2.map(lambda y : ( y[1][1]+":"+((y[1][0]).split(":"))[0]   ,   1 ))
print("(AgeGroup:Occpation:Genre  , 1)")
agor_count.take(6)

(AgeGroup:Occpation:Genre  , 1)
Out[224]: [('18 to 35 :0:Drama', 1),
 ('18 to 35 :0:Action|Adventure|Comedy|Romance', 1),
 ('18 to 35 :0:Action|Adventure|Drama|Sci-Fi|War', 1),
 ('18 to 35 :0:Comedy|Romance|War', 1),
 ('18 to 35 :0:Comedy', 1),
 ("18 to 35 :0:Animation|Children's|Comedy|Musical|Romance", 1)]

In [0]:
count = agor_count.reduceByKey(lambda x,y:x+y)
count.take(6)

Out[225]: [('18 to 35 :0:Action|Adventure|Comedy|Romance', 133),
 ("18 to 35 :0:Animation|Children's|Comedy|Musical|Romance", 47),
 ('18 to 35 :0:Drama|Romance', 757),
 ('18 to 35 :0:Comedy|Musical|Romance', 68),
 ('18 to 35 :0:Comedy|Romance|Thriller', 70),
 ('18 to 35 :0:Action|Adventure|Comedy', 73)]

In [0]:
kpi3_join3 = count.leftOuterJoin(sum).map(lambda y :( y[0]  ,  y[1][1]/y[1][0]))
print("AgeGroup : Occupation : Genre  , Avg .rating")
kpi3_join3.take(20)

AgeGroup : Occupation : Genre  , Avg .rating
Out[231]: [('18 to 35 :0:Action|Adventure|Comedy|Romance', 3.8045112781954886),
 ("18 to 35 :0:Animation|Children's|Comedy|Musical|Romance",
  3.5319148936170213),
 ('18 to 35 :0:Drama|Romance', 3.628797886393659),
 ('18 to 35 :0:Comedy|Musical|Romance', 3.4411764705882355),
 ('18 to 35 :0:Comedy|Romance|Thriller', 3.342857142857143),
 ('18 to 35 :0:Action|Adventure|Comedy', 3.3013698630136985),
 ('18 to 35 :0:Horror|Thriller', 3.0264900662251657),
 ("18 to 35 :0:Children's|Comedy", 3.040983606557377),
 ('18 to 35 :0:Adventure|Comedy|Drama', 4.215686274509804),
 ('18 to 35 :0:Action|Thriller', 3.3742603550295858),
 ("18 to 35 :0:Adventure|Children's|Comedy|Fantasy|Sci-Fi",
  3.2413793103448274),
 ('18 to 35 :0:Action|Comedy|Crime|Drama', 3.3684210526315788),
 ('18 to 35 :4:Horror|Romance', 3.3780487804878048),
 ('18 to 35 :4:Horror|Mystery|Thriller', 2.74468085106383),
 ('18 to 35 :4:Animation|Comedy', 3.8811188811188813),
 ('18 to 35 :14:Do

In [0]:
# Create a view or table
simple_kpi3 = kpi3_join3.map(lambda y :( ((y[0]).split(":"))[0]  ,  ((y[0]).split(":"))[1]  ,   ((y[0]).split(":"))[2]  ,  round(y[1],4) )  )
kpi3 = simple_kpi3.collect()
kpi3_df =pd.DataFrame(kpi3, columns=["AgeGroup","Occupation","Genre","Avg.Rating"])
kpi3_df.sample(25)

,AgeGroup,Occupation,Genre,Avg.Rating
9897,18 to 35,5,Drama|Thriller|War,4.2500
6884,50+,15,Crime|Drama|Film-Noir|Thriller,4.0000
8522,18 to 35,11,Adventure|Thriller,4.6000
6469,36 to 50,1,Drama|Fantasy|Romance|Thriller,4.3333
6477,50+,9,Documentary,2.6667
8924,18 to 35,14,Comedy|Crime|Mystery|Thriller,4.2500
1716,36 to 50,8,Film-Noir|Mystery|Thriller,4.5000
4762,36 to 50,2,Crime|Mystery,2.0000
4390,18 to 35,6,Crime|Drama|Romance|Thriller,4.3750
7253,36 to 50,6,Action|Adventure|Comedy|Romance,3.2222
